In [279]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('listings.csv')

/Users/sharadvrao/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

In [18]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [71]:
relevant_data = data[relevant_features]

In [70]:
relevant_features = ['id', 'listing_url', 'name', 'summary', 'space', 'description', 
                     'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 
                     'house_rules', 'picture_url', 'neighbourhood', 'neighbourhood_cleansed', 
                     'neighbourhood_group_cleansed', 'zipcode', 'latitude', 'longitude', 'property_type', 
                     'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 
                     'square_feet', 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 
                     'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 
                     'number_of_reviews', 'cancellation_policy', 'require_guest_profile_picture', 
                     'require_guest_phone_verification']

In [27]:
reviews = pd.read_csv('reviews.csv')

In [31]:
relevant_data.shape

(8459, 42)

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,picture_url,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,number_of_reviews,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,2318,https://www.airbnb.com/rooms/2318,Casa Madrona -Urban Oasis! *Discount all of Feb*!,"Gorgeous, architect remodeled, 1917 Dutch Colonial in the heart of Madrona. 2 blocks to restaurants and shops, and library. 1 block to playground and park, with doggy play area. 10 mins to Lake WA beaches, trails and Marina. 3 mins to Cap Hill. 4 Bedroom, 2.5 bath, Chef's Kitchen, gracious living and dining room, enclosed back garden, deck, grill, fruit trees. We are flexible with the bedrooms. Kid's toys, books and swing chair! More than 8 people? Check out our Casita next door!","Casa Madrona is a gorgeous, architect remodeled, (Phone number hidden by Airbnb) Dutch Colonial in the heart of Madrona. This beautiful, gracious home has been completely restored from the studs up, adding period touches such as window benches and floor to ceiling bookshelves. The house is perfect for family gatherings, large or small, groups of friends, or colleagues. Upstairs are 4 bedrooms and two full baths with Olympic mountain and city skyline views. The main floor features a formal entry leading to a sun-lit living room with cozy gas fireplace, and lots of books, couches and window seat for cozy relaxing. French doors lead to a spacious, sunny dining room and a huge, open, fully stocked Chef's kitchen with enormous island and even a little cookbook nook. The dining room leads to a large cedar deck and a fully fenced back yard with scrumptious apples in the Autumn, and raspberries and herbs in the summer. The basement has a top of the line, large-sized washer/dryer. There is an","Gorgeous, architect remodeled, (Phone number hidden by Airbnb) Dutch Colonial in the heart of Madrona. 2 blocks to restaurants and shops, and library. 1 block to playground and park, with doggy play area. 10 mins to Lake WA beaches, trails and Marina. 3 mins to Cap Hill. 4 Bedroom, 2.5 bath, Chef's Kitchen, gracious living and dining room, enclosed back garden, deck, grill, fruit trees. We are flexible with the bedrooms. Kid's toys, books and swing chair! More than 8 people? Check out our Casita next door! Casa Madrona is a gorgeous, architect remodeled, (Phone number hidden by Airbnb) Dutch Colonial in the heart of Madrona. This beautiful, gracious home has been completely restored from the studs up, adding period touches such as window benches and floor to ceiling bookshelves. The house is perfect for family gatherings, large or small, groups of friends, or colleagues. Upstairs are 4 bedrooms and two full baths with Olympic mountain and city skyline views. The main floor features","Madrona is a hidden gem of a neighborhood. It has sparkle and charm, and excellent restaurants. It has close proximity to both downtown, AND beautiful, natural trails and woods to Lake WA beaches. After a day in the city, Madrona is a quiet, extremely walkable oasis, with tennis courts, yoga studios, a pub that kids can go to, delicious cup-cakes and ice cream shops, and a playfield for kids and dogs.","We adhere to a 10pm -9am quiet hour schedule, ensuring that our neighbors and other guests are not disturbed by noise during these hours. Thank you in advance for your cooperation and enjoy your stay.",NaN,Guests can access any part of the house.,We are a family who live next door and are available when you need us.,four day minimum stay,https://a0.muscache.com/im/pictures/ac08892c-f4d8-4be4-8e0c-4d1190e8005c.jpg?aki_policy=large,Madrona,Madrona,Central Area,

In [74]:
training_selection = ['id', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 
                      'notes', 'transit', 'access', 'interaction', 'house_rules', 'neighbourhood',
                      'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode', 'latitude',
                      'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
                      'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price',
                      'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
                      'maximum_nights', 'number_of_reviews', 'cancellation_policy',
                      'require_guest_profile_picture', 'require_guest_phone_verification']

In [100]:
training_data = relevant_data[training_selection]

In [101]:
initial_training_data = training_data[training_data.columns[~training_data.isnull().any()]]

In [104]:
clean_training['require_guest_profile_picture'] = (
    initial_training_data['require_guest_profile_picture']=='t').astype(int)

/Users/sharadvrao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [103]:
clean_training = initial_training_data

In [109]:
clean_training['require_guest_phone_verification'] = (
    initial_training_data['require_guest_phone_verification']=='t').astype(int)

/Users/sharadvrao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [117]:
clean_training['price'] = clean_training['price'].apply(lambda x: float(x.replace('$', '').replace(',', '')))

/Users/sharadvrao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [119]:
clean_training['extra_people'] = clean_training['extra_people'].apply(
    lambda x: float(x.replace('$', '').replace(',', '')))

/Users/sharadvrao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [125]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_training['name'])

In [134]:
X.shape

(8459, 3005)

In [142]:
clean_training['cancellation_policy'].unique()

array(['strict_14_with_grace_period', 'moderate', 'flexible',
       'super_strict_60', 'strict', 'super_strict_30'], dtype=object)

In [500]:
clean_training['bed_type'].unique()

array(['Real Bed', 'Futon', 'Pull-out Sofa', 'Airbed', 'Couch'],
      dtype=object)

In [146]:
clean_training['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [149]:
clean_training['is_home/apt'] = (clean_training['room_type'] == 'Entire home/apt').astype(int)

In [151]:
clean_training['is_private_room'] = (clean_training['room_type'] == 'Private room').astype(int)

In [152]:
clean_training['is_shared_room'] = (clean_training['room_type'] == 'Shared room').astype(int)

In [155]:
clean_training['property_type'].unique()

array(['House', 'Guest suite', 'Apartment', 'Condominium', 'Guesthouse',
       'Bed and breakfast', 'Townhouse', 'Bungalow', 'Cottage', 'Boat',
       'Loft', 'Cabin', 'Serviced apartment', 'Houseboat', 'Camper/RV',
       'Other', 'Tiny house', 'Treehouse', 'Hostel', 'Tent', 'Yurt',
       'Aparthotel', 'Chalet', 'Villa', 'Boutique hotel', 'Farm stay',
       'Nature lodge', 'Resort', 'In-law'], dtype=object)

In [157]:
clean_training['bed_type'].unique()

array(['Real Bed', 'Futon', 'Pull-out Sofa', 'Airbed', 'Couch'],
      dtype=object)

In [158]:
clean_training['has_real_bed'] = (clean_training['bed_type'] == 'Real Bed').astype(int)

In [159]:
clean_training['has_futon'] = (clean_training['bed_type'] == 'Futon').astype(int)

In [160]:
clean_training['has_pullout'] = (clean_training['bed_type'] == 'Pull-out Sofa').astype(int)

In [161]:
clean_training['has_airbed'] = (clean_training['bed_type'] == 'Airbed').astype(int)

In [162]:
clean_training['has_couch'] = (clean_training['bed_type'] == 'Couch').astype(int)

In [163]:
clean_training

,id,name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,is_home/apt,is_private_room,is_shared_room,has_real_bed,has_futon,has_pullout,has_airbed,has_couch
0,2318,Casa Madrona -Urban Oasis! *Discount all of Feb*!,Madrona,Central Area,47.610819,-122.290816,House,Entire home/apt,8,4,Real Bed,"{Internet,Wifi,Kitchen,""Free parking on premises"",""Pets allowed"",""Free street parking"",""Indoor fireplace"",Heating,""Family/kid friendly"",Washer,Dryer,""Smoke detector"",""Carbon monoxide detector"",""First aid kit"",""Fire extinguisher"",Essentials,Shampoo,Hangers,""Hair dryer"",Iron,""Laptop friendly workspace"",""High chair"",""Children’s books and toys"",""Pack ’n Play/travel crib"",""Hot water"",""Luggage dropoff allowed"",""Long term stays allowed"",""Host greets you"",""Lake access""}",296.0,8,25.0,7,300,21,strict_14_with_grace_period,0,0,1,0,0,1,0,0,0,0
1,4291,Sunrise in Seattle Master Suite,Roosevelt,Other neighborhoods,47.687801,-122.313427,House,Private room,2,1,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Free parking on premises"",Heating,Washer,Dryer,""Smoke detector"",""Fire extinguisher"",Essentials,Shampoo}",82.0,1,0.0,2,60,63,moderate,0,0,0,1,0,1,0,0,0,0
2,5682,"Cozy Studio, min. to downtown -WiFi",South Delridge,Delridge,47.523980,-122.359891,Guest suite,Entire home/apt,2,0,Real Bed,"{TV,Internet,Wifi,""Free street parking"",Heating,""Family/kid friendly"",""Smoke detector"",""Carbon monoxide detector"",""First aid kit"",Essentials,Shampoo,""24-hour check-in"",Hangers,""Hair dryer"",Iron,""translation missing: en.hosting_amenity_49"",""Self check-in"",Lockbox,""Private entrance"",""Hot water"",Microwave,""Coffee maker"",Refrigerator,""Single level home"",""Flat path to front door""}",48.0,1,5.0,3,14,462,strict_14_with_grace_period,0,1,1,0,0,1,0,0,0,0
3,6606,"Fab, private seattle urban cottage!",Wallingford,Other neighborhoods,47.654109,-122.337605,Apartment,Entire home/apt,2,1,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free parking on premises"",Heating,""Smoke detector"",""Carbon monoxide detector"",""Fire extinguisher"",Essentials,Shampoo,""24-hour check-in"",Hangers,""Hair dryer"",Iron,""Laptop friendly workspace"",""translation missing: en.hosting_amenity_50""}",90.0,2,10.0,2,1125,134,strict_14_with_grace_period,0,0,1,0,0,1,0,0,0,0
4,9419,Glorious sun room w/ memory foambed,Georgetown,Other neighborhoods,47.550620,-122.320135,Apartment,Private room,2,1,Real Bed,"{Internet,Wifi,""Air conditioning"",Kitchen,""Free parking on premises"",""Pets live on this property"",Dog(s),""Free street parking"",Heating,Washer,Dryer,""Smoke detector"",""First aid kit"",""Safety card"",""Fire extinguisher"",Essentials,Shampoo,""Lock on bedroom door"",Hangers,""Hair dryer"",Iron,""Laptop friendly workspace"",""Private living room"",""Hot water"",""Bed linens"",""Extra pillows and blankets"",Microwave,""Coffee maker"",Refrigerator,""Dishes and silverware"",""Cooking basics"",Oven,Stove,""Host greets you""}",65.0,1,5.0,2,60,131,moderate,1,1,0,1,0,1,0,0,0,0
5,9460,Downtown Convention Center B&B -- Free Minibar,First Hill,Downtown,47.608010,-122.328739,Condominium,Private room,2,1,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning"",Kitchen,Doorman,Breakfast,Elevator,""Free street parking"",""Buzzer/wireless intercom"",Heating,""Family/kid friendly"",Washer,Dryer,""Smoke detector"",""Carbon monoxide detector"",""First aid kit"",""Safety card"",""Fire extinguisher"",Essentials,Shampoo,""Lock on bedroom door"",""24-hour check-in"",Hangers,""Hair dryer"",Iron,""Laptop friendly workspace"",""Self check-in"",""Smart lock"",""Room-darkening shades"",""Hot water"",""Body soap"",""Bath towel"",""Toilet paper"",""Bed linens"",""Extra pillows and blankets"",Microwave,""Coffee mak

In [165]:
clean_training['cancellation_policy'].unique()

array(['strict_14_with_grace_period', 'moderate', 'flexible',
       'super_strict_60', 'strict', 'super_strict_30'], dtype=object)

In [169]:
first_run = clean_training.iloc[:,[0, 8, 9, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]

In [185]:
first_run_norm = first_run.copy()

In [186]:
scaler = StandardScaler()
first_run_norm['accommodates'] = scaler.fit_transform(first_run['accommodates'].values.reshape(-1,1))

In [187]:
scaler = StandardScaler()
first_run_norm['bedrooms'] = scaler.fit_transform(first_run['bedrooms'].values.reshape(-1,1))

In [188]:
scaler = StandardScaler()
first_run_norm['price'] = scaler.fit_transform(first_run['price'].values.reshape(-1,1))

In [189]:
scaler = StandardScaler()
first_run_norm['guests_included'] = scaler.fit_transform(first_run['guests_included'].values.reshape(-1,1))

In [190]:
scaler = StandardScaler()
first_run_norm['extra_people'] = scaler.fit_transform(first_run['extra_people'].values.reshape(-1,1))

In [191]:
scaler = StandardScaler()
first_run_norm['minimum_nights'] = scaler.fit_transform(first_run['minimum_nights'].values.reshape(-1,1))

In [192]:
scaler = StandardScaler()
first_run_norm['maximum_nights'] = scaler.fit_transform(first_run['maximum_nights'].values.reshape(-1,1))

In [193]:
scaler = StandardScaler()
first_run_norm['number_of_reviews'] = scaler.fit_transform(first_run['number_of_reviews'].values.reshape(-1,1))

In [272]:
first_run_norm

,id,accommodates,bedrooms,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,require_guest_profile_picture,require_guest_phone_verification,is_home/apt,is_private_room,is_shared_room,has_real_bed,has_futon,has_pullout,has_airbed,has_couch
0,2318,1.910552,2.590422,0.828355,3.555118,0.608606,0.262646,-0.537525,-0.363893,0,0,1,0,0,1,0,0,0,0
1,4291,-0.721020,-0.383436,-0.440030,-0.592111,-0.621220,-0.125604,-0.979090,0.276910,0,0,0,1,0,1,0,0,0,0
2,5682,-0.721020,-1.374723,-0.641549,-0.592111,-0.375255,-0.047954,-1.063724,6.364540,0,1,1,0,0,1,0,0,0,0
3,6606,-0.721020,-0.383436,-0.392613,0.000350,-0.129289,-0.125604,0.980356,1.360172,0,0,1,0,0,1,0,0,0,0
4,9419,-0.721020,-0.383436,-0.540789,-0.592111,-0.375255,-0.125604,-0.979090,1.314401,1,1,0,1,0,1,0,0,0,0
5,9460,-0.721020,-0.383436,-0.457811,-0.592111,0.116676,-0.047954,-1.036126,5.510136,0,0,0,1,0,1,0,0,0,0
6,9531,0.156171,0.607850,0.051914,0.000350,-0.129289,-0.125604,-0.417934,-0.135035,0,1,1,0,0,1,0,0,0,0
7,9534,-0.282424,0.607850,-0.185167,0.000350,-0.375255,-0.125604,-0.417934,-0.135035,0,1,1,0,0,1,0,0,0,0
8,9596,0.156171,-0.383436,-0.214803,0.000350,0.116676,-0.125604,-0.979090,0.475254,0,0,1,0,0,1,0,0,0,0
9,9909,1.471957,1.599136,-0.185167,1.185273,-0.621220,-0.047954,-0.868699,0.185366,0,0,1,0,0,1,0,0,0,0


In [243]:
test = np.zeros(19)

In [244]:
test[3] = 100

In [249]:
results = cosine_similarity(test.reshape(-1,1).T, test2.reshape(-1,1).T)

In [234]:
np.argsort(results)

array([[0]])

In [250]:
results

array([[0.98058068]])

In [229]:
results

array([[ 3.57356792e-04, -1.02547120e-04, -1.12908898e-04, ...,
         2.16492251e-08, -1.95301524e-08,  1.72347963e-08]])

In [245]:
test2 = test.copy()

In [246]:
test2[0] = 20

In [287]:
def cos_sim(data, row):
    norms = np.linalg.norm(data, axis=1)
    row_norm = np.linalg.norm(row)
    inner_prods = np.inner(row, data)
    sims = inner_prods / (row_norm * norms)
    return inner_prods
    

In [257]:
a = np.array([1, 3, 4])

In [258]:
b = np.array([[2, 1, 6], [5, 7, 8]])

In [261]:
np.inner(a,b)

array([29, 58])

In [289]:
np.argmax(cos_sim(first_run_norm, test))

7213

In [269]:
test = test*10

In [278]:
first_run_norm.iloc[0,:]

id                                  2318.000000
accommodates                        1.910552   
bedrooms                            2.590422   
price                               0.828355   
guests_included                     3.555118   
extra_people                        0.608606   
minimum_nights                      0.262646   
maximum_nights                     -0.537525   
number_of_reviews                  -0.363893   
require_guest_profile_picture       0.000000   
require_guest_phone_verification    0.000000   
is_home/apt                         1.000000   
is_private_room                     0.000000   
is_shared_room                      0.000000   
has_real_bed                        1.000000   
has_futon                           0.000000   
has_pullout                         0.000000   
has_airbed                          0.000000   
has_couch                           0.000000   
Name: 0, dtype: float64

In [286]:
cosine_similarity(first_run_norm.iloc[0,:].values.reshape(-1,1), test.reshape(-1,1))

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.

In [283]:
first_run_norm.iloc[0,:].values.reshape(-1,1)

array([[ 2.31800000e+03],
       [ 1.91055178e+00],
       [ 2.59042250e+00],
       [ 8.28355090e-01],
       [ 3.55511833e+00],
       [ 6.08605914e-01],
       [ 2.62645775e-01],
       [-5.37525030e-01],
       [-3.63893436e-01],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 1.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 1.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])

In [284]:
test.reshape(-1,1)

array([[   0.],
       [   0.],
       [   0.],
       [1000.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.]])

In [301]:
inputs = ['2014-03-25', '2012-08-05']

In [326]:
out = []
for each in inputs:
    curr_string = each.split('-')
    if int(curr_string[1]) >= 1 and int(curr_string[1]) <= 3
        curr_quarter = 'Q1'
    elif 

In [325]:
curr_string

['2012', '08', '05']

In [460]:
inputs = [2, 3, 5, 1, 1, 2, 1]

In [333]:
while(curr_sum <= total):

NameError: name 'curr_sum' is not defined

In [484]:
possible = [[]]*10

In [485]:
curr_sum = 0
possible = []
count = 0
now = 0
for index, current in enumerate(inputs):
    curr_sum += current
    if curr_sum < total:
        possible.append(current)
        inputs.remove(inputs[index])
    else:
        now += 1
        
        
    

In [486]:
possible

[]

In [477]:
possible

[[3], [], [], [], [], [], [], [], [], []]

In [347]:
a.remove(a[0])

In [348]:
a

[4, 5, 3]

In [498]:
def subset_sum(numbers, target, partial=[]):
    outs = []
    s = sum(partial)

    if (s == target) and (np.isin(numbers, partial).all()):
        outs.append(len(partial))
    if s >= target:
        return

    for index in range(len(numbers)):
        n = numbers[index]
        remaining = numbers[index+1:]
        subset_sum(remaining, target, partial + [n]) 
    return outs

In [499]:
subset_sum([2, 3, 5, 1, 1, 2, 1], 5)

[]

In [452]:
a = subset_sum

In [384]:
a

<function __main__.subset_sum(numbers, target, partial=[])>

In [394]:
a = [1, 2, 3]

In [395]:
inputs

[2, 3, 5, 1, 1, 2, 1]

In [396]:
a in inputs

False

In [415]:
a = [2, 3]

In [414]:
a in inputs

False

In [406]:
curr_sum = 0
curr = 0
while curr_sum < total:
    curr_sum += inputs[curr]
    
    

[5]

In [430]:
inputs

[2, 3, 5, 1, 1, 2, 1]

In [432]:
np.isin(inputs, a).all()

False

In [429]:
a = [5]

In [459]:
import math
pi = math.pi

def PointsInCircum(r,n=100):
    return [(math.cos(2*pi/n*x)*r,math.sin(2*pi/n*x)*r) for x in range(0,n+1)]